<a href="https://colab.research.google.com/github/sefeoglu/AE_Parseval_Network/blob/master/src/notebooks/FGSM_WResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import sys
sys.path.insert(0, os.path.abspath('../'))

In [0]:
from sklearn.model_selection import train_test_split
import gzip
import pickle
import numpy as np

In [4]:
def read_data():
    with open("data.pz", 'rb') as file_:
        with gzip.GzipFile(fileobj=file_) as gzf:
            data = pickle.load(gzf, encoding='latin1', fix_imports=True)
    return data
data = read_data()
new_data_X = []
Y_data = []
for row in data:
    new_data_X.append(row['crop'])
    Y_data.append(row['label'])
new_data_X = np.array(new_data_X)
new_data_X.shape

(5722, 68, 100)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(new_data_X, Y_data, test_size=0.33, random_state=42)

In [0]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [0]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
# creating initial dataframe

y_train_df = pd.DataFrame(y_train, columns=['Label'])
# creating instance of labelencoder
labelencoder = LabelEncoder()
# Assigning numerical values and storing in another column
y_train_df['New'] = labelencoder.fit_transform(y_train_df['Label'])
y_test_df = pd.DataFrame(y_test, columns=['Label'])
# creating instance of labelencoder
labelencoder = LabelEncoder()
# Assigning numerical values and storing in another column
y_test_df['New'] = labelencoder.fit_transform(y_test_df['Label'])

In [8]:
import keras.backend as K
img_rows, img_cols = X_train[0].shape


# transform data set
if K.common.image_data_format() == 'channels_first':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

Using TensorFlow backend.


In [0]:
from keras.utils import to_categorical
y_test = to_categorical(y_test_df['New'])
y_train = to_categorical(y_train_df['New'])

In [0]:
import keras

model = keras.models.load_model("wresnet.h5")

Adversarial Attack


In [19]:
model.input.shape

TensorShape([None, 68, 100, 1])

In [0]:
def preprocess(image):
  image = tf.cast(image, tf.float32)
  image = tf.image.resize(image, (68, 100))
  image = tf.keras.applications.mobilenet_v2.preprocess_input(image)
  image = image[None, ...]
  return image

# Helper function to extract labels from probability vector

In [0]:
img = X_train[0]

In [0]:
import tensorflow as tf
image = tf.convert_to_tensor(img)
image = preprocess(image)

array([[0.02913386, 0.21254967, 0.10889897, 0.64941746]], dtype=float32)

In [116]:
from keras.layers import Model
layer_outputs = [layer.output for layer in model.layers]
viz_model = Model(input=model.get_input_at(4), output=layer_outputs)
features = viz_model(image)
for a in features:
  print(a)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=tf.Tensor(..., outputs=[<tf.Tenso...)`
  This is separate from the ipykernel package so we can avoid doing imports until


ValueError: ignored

In [55]:
probs = model.predict(image,batch_size=None,steps=1)
labrador_retriever_index = 208
label = tf.one_hot(0, probs.shape[-1])
label = tf.reshape(label, (1, probs.shape[-1]))
label

<tf.Tensor: shape=(1, 4), dtype=float32, numpy=array([[1., 0., 0., 0.]], dtype=float32)>

In [60]:
probs = tf.convert_to_tensor(probs)

<tf.Tensor: shape=(1, 4), dtype=float32, numpy=array([[0.02913386, 0.21254967, 0.10889897, 0.64941746]], dtype=float32)>

90112/83281 [================================] - 0s 0us/step


In [73]:
loss_object = tf.keras.losses.CategoricalCrossentropy()
with tf.GradientTape() as tape:
    tape.watch(image)
    loss = loss_object(label, probs)

  # Get the gradients of the loss w.r.t to the input image.
gradient = tape.gradient(loss,image)
  # Get the sign of the gradients to create the perturbation
print(image.shape)

(1, 224, 224, 3)
